In [1]:
# Set notebook to use only one GPU
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


# Results Analysis

Analyse results


In [2]:
from file_functions import *
from models_bachelors import *
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
from sklearn.metrics import accuracy_score
from plotly.subplots import make_subplots
import plotly.graph_objects as go

mcdropconnect = {'lockbox': {'labels': 0,
                             'preds': 0},
                 'test':    {'labels': 0,
                             'preds': 0}}
mcdropout = {'lockbox': {'labels': 0,
                             'preds': 0},
                 'test':    {'labels': 0,
                             'preds': 0}}
standard = {'lockbox': {'labels': 0,
                             'preds': 0},
                 'test':    {'labels': 0,
                             'preds': 0}}

# Both are dicts with keys: 'lockbox_preds_lst' and 'preds_lst'. With
with h5py.File('mcdropconnect/predictions_mcdropconnect.h5', 'r') as file: 
    for group in file.keys():
        for dset in file[group].keys():
            mcdropconnect[dset][group] = file[group][dset][()]        # Get dataset as a numpy array. Yes, I know, the order is reversed...

# Both are dicts with keys: 'lockbox_preds_lst' and 'preds_lst'. With
with h5py.File('mcdropout/predictions_mcdropout.h5', 'r') as file: 
    for group in file.keys():
        for dset in file[group].keys():
            mcdropout[dset][group] = file[group][dset][()]        # Get dataset as a numpy array. Yes, I know, the order is reversed...
            
            
# Both are dicts with keys: 'lockbox_preds_lst' and 'preds_lst'. With
with h5py.File('predictions_standard.h5', 'r') as file: 
    for group in file.keys():
        for dset in file[group].keys():
            standard[dset][group] = file[group][dset][()]   

Using TensorFlow backend


Keras Uncertainty will use standalone Keras backend

# Rejection use case
Compute predictive entropy over all subjects for both methods (maybe normalise it). Then make a plot of coverage vs accuracy (threshold) for both methods. Because the predictions are alreaady averaged forward passes, the pred. entropy of a single input of shape (4,) would be simply the sum of pc * log(pc) for all the class probs.

In [9]:
# Avg predictive entropy on test set for both methods
acc_mcdropout = []
acc_mcdropconnect = []
acc_standard = []

methods = {'mcdropconnect': mcdropconnect, 'mcdropout':mcdropout, 'standard': standard}

for name, method in methods.items():
    print(f'{name.upper()}')
    entropy = predictive_uncertainty(method['test']['preds'])    # shape: (9,576)
    Y_true = method['test']['labels']    # shape: (9,576,4)
    # corrects = np.argmax(Y_true, axis=-1) == np.argmax(method['test']['preds'], axis=-1)    # axis -1 if (9,576,4). Also get corrects across ALL subjects
    # entropy_correct = entropy[corrects]
    # entropy_wrong = entropy[~corrects]
    trues = np.argmax(Y_true, axis=-1)
    preds = np.argmax(method['test']['preds'], axis=-1)
    # # For distribution plots of predictive entropy
    # hist_data = [entropy_correct, entropy_wrong]
    # group_labels = ['Correct', 'Incorrect']
    # fig = ff.create_distplot(hist_data, group_labels, bin_size=.05)
    # fig.show()
    fig = make_subplots(rows=3,cols=3, subplot_titles=[f'Subj.{x}' for x in range(0,9)])
    for r in range(1,4):
        for col in range(1,4):
            # World's most convoluted formula for finding out the cell number istg
            subj_id = (3 * r) - 4 + col
            corrects = np.argmax(Y_true[subj_id], axis=-1) == np.argmax(method['test']['preds'][subj_id], axis=-1)
            entropy_correct = entropy[subj_id][corrects]
            entropy_wrong = entropy[subj_id][~corrects]
            hist_data = [entropy_correct, entropy_wrong]
            group_labels = ['Corr.', 'Incorr.']
            sub = ff.create_distplot(hist_data, group_labels, bin_size=0.05)
            for trace in sub.select_traces(3):
                fig.add_trace(trace, row=r, col=col)
            for trace in sub.select_traces(2):
                fig.add_trace(trace, row=r, col=col)

    fig.show()
    for idx, subject in enumerate(trues):
        acc = accuracy_score(y_true=subject, y_pred=preds[idx], normalize=True)
        if name == 'mcdropconnect':
            acc_mcdropconnect.append(acc)
        elif name == 'mcdropout':
            acc_mcdropout.append(acc)
        else:
            acc_standard.append(acc)

print(f'acc_mcdropconnect:{np.mean(acc_mcdropconnect)}')
print(f'acc_mcdropout:{np.mean(acc_mcdropout)}')
print(f'acc_std:{np.mean(acc_standard)}')
        

MCDROPCONNECT


MCDROPOUT


STANDARD


acc_mcdropconnect:0.47608024691358025
acc_mcdropout:0.48996913580246915
acc_std:0.48919753086419754


In [3]:

# I wanna see what entropy is like for a standard model
entropy = uncertainty(Y_preds)
corrects = np.argmax(Y_true, axis=1) == np.argmax(Y_preds, axis=1)
entropy_correct = entropy_preds[corrects]
entropy_wrong = entropy_preds[~corrects]

# Combined distribution of entropy for correct and incorrect predictions
hist_data = [entropy_correct, entropy_wrong]
group_labels = ['Correct', 'Incorrect']

fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()
            

NameError: name 'uncertainty' is not defined